In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [101]:
MotData = pd.read_csv("ER - Motion Values and Attack Data (App Ver. 1.08) - Motion Values.csv")
PseData = pd.read_csv("ER - Motion Values and Attack Data (App Ver. 1.08) - Poise Damage - PvP.csv")

## Weapon Data

Elden Ring has 377 distinct weapons, and each weapon has a number of different attacks it can do based on different button inputs combined with context. The dataset's rows correspond to the weapons, and its columns correspond to each specific kind of attack. The dataset consists of over a dozen separate spreadsheets which have values that correspond to each attack. We will examine "Motion Values" and "Poise Damage - PvP". The former determines what percentage change to a weapon's base damage a particular move should have, with slower moves that deal more damage and critical hits dealing massive amounts of damage. The latter refers to how good attacks are at staggering other players in "Player versus Player" combat, with player poise being measured on a scale of 0-133.

In [42]:
sum(MotData["Weapon"].notnull())

377

In [136]:
MotData

,Weapon Class,Weapon,1h R1 1,1h R1 2,1h R1 3,1h R1 4,1h R1 5,1h R1 6,1h R2 1,1h R2 2,...,Mounted R1 3,Mounted R2,Mounted R2 Charging,Mounted Charged R2,Mounted L1 1,Mounted L1 2,Mounted L1 3,Mounted L2,Mounted L2 Charging,Mounted Charged L2
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Dagger,Dagger,100.0,101.0,102.0,103.0,104.0,110.0,120.0,120,...,110.0,135.0,90.0,170,110.0,110.0,110.0,135.0,90.0,170
2,Dagger,Parrying Dagger,100.0,101.0,102.0,103.0,104.0,110.0,120.0,120,...,110.0,135.0,90.0,170,110.0,110.0,110.0,135.0,90.0,170
3,Dagger,Miséricorde,100.0,101.0,102.0,103.0,104.0,110.0,120.0,120,...,110.0,135.0,90.0,170,110.0,110.0,110.0,135.0,90.0,170
4,Dagger,Great Knife,100.0,101.0,102.0,103.0,104.0,110.0,120.0,120,...,110.0,135.0,90.0,170,110.0,110.0,110.0,135.0,90.0,170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,Greatshield,Ant's Skull Plate,100.0,102.0,105.0,NaN,NaN,NaN,125.0,125,...,110.0,125.0,100.0,165,110.0,110.0,110.0,125.0,100.0,165
407,Greatshield,Erdtree Greatshield,100.0,102.0,105.0,NaN,NaN,NaN,125.0,125,...,110.0,125.0,100.0,165,110.0,110.0,110.0,125.0,100.0,165
408,Greatshield,Jellyfish Shield,100.0,102.0,105.0,NaN,NaN,NaN,125.0,125,...,110.0,125.0,100.0,165,110.0,110.0,110.0,125.0,100.0,165
409,Greatshield,Visage Shield,100.0,102.0,105.0,NaN,NaN,NaN,125.0,125,...,110.0,125.0,100.0,165,110.0,110.0,110.0,125.0,100.0,165


In [43]:
WeaponNames = MotData["Weapon"].to_numpy()

In [10]:
np.where(WeaponNames == "Godskin Peeler")[0][0]

122

In [44]:
MotData.size

29592

In [45]:
MotData.shape

(411, 72)

### Parsing Data: multi attacks

Some weapons in Elden Ring do multiple separate attacks with a single input

In [110]:
def stringconv(input):
    if(type(input) != str):
        return float(input)
    if(input.replace(".", "", 1).isdigit()):
        return float(input)
    sum = 0.0
    tempstr = ""
    for x in input:
        if x.isdigit() or x == '.':
            tempstr+=x
            continue
        if tempstr == "":
            continue
        sum += float(tempstr)
        tempstr = ""
    if tempstr == "":
        return sum
    sum += float(tempstr)
    tempstr = ""
    return sum

In [111]:
stringconv("103294.98")

103294.98

In [112]:
stringconv("115 + (57.5 Projectile) + 230 + (57.5 Projectile)")

460.0

In [135]:
MotData["1h R2 1"] = MotData["1h R2 1"].apply(stringconv)
PseData["1h R2 1"] = PseData["1h R2 1"].apply(stringconv)
MotData["1h Running R1"] = MotData["1h Running R1"].apply(stringconv)
PseData["1h Running R1"] = PseData["1h Running R1"].apply(stringconv)
MotData["1h Running R2"] = MotData["1h Running R2"].apply(stringconv)
PseData["1h Running R2"] = PseData["1h Running R2"].apply(stringconv)

## Poise Breakpoints

### Poise

Poise is a value from 0 to 133 which determines how hard it is for enemy attacks to stagger your player character, breifly preventing you from taking more actions and giving a small window for unimpeded enemy attacks. Each attack deals a certain amount of "poise damage", and when your character has been dealt enough poise damage, they will stagger. If the player has not been dealt poise damage for 60 seconds or gets staggered, the stored amount of poise damage will reset to zero.

Your character's armor, separated into headgear, chest, gauntlets, and leggings, will contribute 0-100 poise in total. The highest poise armor sets are the "Bull-Goat Set" giving 100 poise, the "Fire Prelate Set" at 96 poise, the "Omen Set" at 91 poise, "Lionel's Set" at 86 poise, and the "Veteran's Set" at 80 poise. In addition there is the "Bull-Goat's Talisman" which reduces incoming poise damage by 25%, which the game displays by saying you have 33% more poise, bringing the maximum to 133.

There are temporary effects which increase poise, but as you will see we don't need to account for them in the following analysis. First, Weapon Skill "Endure" will give you infinite poise for 3.5 seconds. The Flask of Woundrous Physick can hold two crystal tears, and can only be used once per fight, so "Leaden Hardtear" gives infinite poise for 10 seconds. The consumable "Ironjar Aromatic" greatly slows the player's speed and makes attacks bounce off of them, at the cost of slowing the player to a crawl. Of little import is (Radiant) Baldachin's Blessing increases poise by 50% (81%) for 15 (70) seconds, these items being far too tedious to obtain repeatedly to see use.

### Limitations of higher poise

Why wouldn't all players simply have 133 poise? The answer is multifaceted. Higher poise armor will be heavier, which has a complex statistical interplay, players may choose armor for fashion rather than poise, and players have to find sets of armor out in the world. 

#### Equip Load
Players have the statistic "Equip Load" which determines how much the sum total of held weapons, armor, and talismans will affect them. Being below 30%, 70%, and 100% or above 100% equip load constitute four tiers of mobility, with lighter tiers granting faster actions and especially faster dodge rolls, while the heaviest tier restricts player to a slow walk and disables dodge rolling. Equip Load scales with the player statistic "Endurance" which also governs stamina i.e. how many actions a player can do at once. Increasing Endurance comes at the cost of increasing statistics which make the player deal more damage, Vigor which governs player health, and Mind which governs mana i.e. how many spells and weapon arts a player can cast. Thus there is a tradition of "tank" builds wearing the heaviest armor and using slower but high damage weapons, while sorcerers tend to wear lighter armor and invest into other stats so their spells can deal absurd amounts of damage. Most players aim to have just below 70% equip load to be in the medium tier and have a decent amount of armor. 

#### Fashion
Secondarily, fashion is another major factor in what armor players use. The Bull-Goat set and the Omen set are considered ugly by many, turning them off from using it. The Fire Prelate chestpiece and helmet also have a very specific look that not all players would want, but the leggings and gauntlets can blend nicely into most attires. Alternatively, players may just have a fondness for how particular pieces of armor look, or they may be "cosplaying" as a particular enemy in the game by using their armor or as a particular character from outside media.

#### Non-poise benefits
Poise is far from the only statistic of armor. Each piece of armor has varying resistances for different types of damage, and increases the player's resistances to certain status effects (bleed, scarlet rot, madness, etc.) in varying amounts. In addition, many helmets grant other benefits like boosting the aforementioned player statistics, and some sets of armor make thematically associated spells stronger (which mechanically encourages cosplaying as certain enemies).

#### Obtaining armor
The higher poise armors are the hardest to obtain, two of the five I listed require you to get into the late game area of the Capital, and a further two require you to complete that area. This leaves the Bull-Goat set as the one you have earliest access in theory, but it requires your character to join a murder cult and assassinate the NPC "Great Horned Targoth" who wears the armor. Furthermore, this quest can be locked out by either killing the Demigod leader of the cult, upon which it dissolves, or choosing to kill Patches when he appears as a boss early in the game, who asks you to spare him at half health. The Lionel's Set is in a very obvious location, so most players will obtain it, and the Fire Prelate Set is a random drop from a strong enemy that players must go past and can kill as many times as they like until they get the complete set.

#### Talismans
Players can use up to four talismans at once, and one of these can be the Bull-Goat's Talisman. The game has many talismans useful to all players, and many that benefit specific builds, so the choice of whether or not to use that specific talisman is significant, and among talismans it has a low equip load cost. Players may opt into using it if they are using lighter armor for the aforementioned reasons, or opt out of using it because they prefer other talismans. The item is relatively easy to obtain.


### Breakpoints
All of this exposition is to say, players want to know what values of poise they can achieve which would allow them to withstand a wide variety of attacks. The community refers to "Poise Breakpoints" as levels of poise which will protect players from being staggered by a certain class of attack. 

#### PvE breakpoints
Enemies all deal either 50 or 120 poise damage with every attack. In addition there are hostile NPCs that will deal the same highly variable levels of poise damage damage we will discuss next. Thus for most "Player versus Environment" combat, it really only matters whether players at 51 poise or above, at 101 poise or above (to sustain two 50 poise attacks) or at 121-133 poise (only the three strongest sets of armor with Bull-Goat's Talisman grant this much poise). 

#### PvP breakpoints


We want to guess what the best poise values a player can have in PvP to be able to withstand a high proportion of attacks without being staggered.

In [130]:
PseArray = np.concatenate((np.array(PseData["1h R1 1"].dropna()), np.array(PseData["1h R2 1"].dropna())))

In [134]:
np.percentile(PseArray, 2)

40.5